    Wayne Nixalo  -  20 May 2017
FAI1 - Practical Deep Learning I - Week 3 HW: Kaggle StateFarm Distracted Driver Competition

#### Imports

In [ ]:
import os, sys
import numpy as np
from glob import glob

In [ ]:
# will need this to access any libraries in superdirectories
sys.path.insert(1, os.path(os.getcwd(), '../utils'))
from vgg16 import Vgg16

#### 1. Run this the First Time Only

Download the Data and get it into the right directories. Fortunately SF already organized the data, so it's just a matter of assigning path variables. NOTE: kaggle-cli needs to be set up beforhand. Also path vars must be assigned each time this notebook is run.

In [ ]:
HOME_DIR = os.getcwd()
DATA_PATH = HOME_DIR + '/data'
TRAIN_PATH = DATA_PATH + '/train'
VAL_PATH = DATA_PATH + '/valid'
TEST_PATH = DATA_PATH + '/test'

In [ ]:
# create the validation directories
os.mkdir(VAL_PATH)
for i in xrange(10):
    os.mkdir(VAL_PATH + '/c' + str(i))

# # another way to do this:
# %mkdir $VAL_PATH
# for i in xrange(10):
#     %mkdir $VAL_PATH/c"$i"

#### 2. Run this if you don't have an Accurate Validation Set

Grab a random permutation from the training data for validation. Do this until validation accuracy matches test accuracy. Also see: http://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python

In [ ]:
%cd $DATA_PATH

VAL_PORTION = 0.2
for i in xrange(10):
    %cd c"$i"
    g = glob('*.jpg')
    shuff = np.random.permutation(g)
    for n in xrange(number * VAL_PORTION):
        os.rename(shuff[n], VAL_PATH + shuff[n])
    % cd ..